# simulation

In [12]:
from utils import (load_problem_data,
                   load_solution)
from evaluation import get_known, get_evaluation
import evaluation_nofailure
import evaluation_failure
import evaluation
import pandas as pd
import numpy as np

demand, datacenters, servers, selling_prices, elasticity = load_problem_data()

In [23]:



def parse_action_string(action_string):
    server_generation, actions, action_params = action_string.split("|")
    actions = actions.split("-")
    action_params = action_params.split("-")
    action_comb = [parse_action_comb_param(server_generation, action, action_param) for action, action_param in zip(actions, action_params)]
    action_comb = [action for action in action_comb if action is not None]
    return action_comb
    
def parse_action_comb_param(server_generation, action, action_param):
    if action == "buy":
        datacenter_id, average_U = action_param.split(",")
        average_U = float(average_U)
        return {"action": action, "datacenter_id": datacenter_id, "average_U": average_U, "server_generation": server_generation}
    elif action == "dismiss":
        dismiss_age = int(action_param)
        return {"action": action, "dismiss_age": dismiss_age, "server_generation": server_generation}
    elif action == "move":
        datacenter_id, average_U, move_age = action_param.split(",")
        average_U = float(average_U)
        move_age = int(move_age)
        return {"action": action, "datacenter_id": datacenter_id, "average_U": average_U, "move_age": move_age,}
    return None

action_comb = parse_action_string("GPU.S3|buy-dismiss|DC3,0.5-84")
action_comb

def get_solution_from_action_comb(action_comb):
    solution = []
    for action in action_comb:
        if action["action"] == "buy":
            datacenter_id = action["datacenter_id"]
            server_generation = action["server_generation"]
            solution.append({"time_step": 1, 
                             "datacenter_id": datacenter_id, 
                             "server_generation": server_generation, 
                             "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", 
                             "action": "buy"})
        elif action["action"] == "dismiss":
            dismiss_age = action["dismiss_age"]
            solution.append({"time_step": 1+dismiss_age, 
                             "datacenter_id": datacenter_id, 
                             "server_generation": server_generation, 
                             "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", 
                             "action": "dismiss"})
        elif action["action"] == "move":
            datacenter_id = action["datacenter_id"]
            move_age = action["move_age"]
            solution.append({"time_step": 1+move_age, 
                             "datacenter_id": datacenter_id, 
                             "server_generation": server_generation, 
                             "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", 
                             "action": "move"})
    solution = pd.DataFrame(solution)
    return solution
solution = get_solution_from_action_comb(action_comb)


In [15]:
df_servers = pd.read_csv('./data/servers.csv')
df_datacenters = pd.read_csv('./data/datacenters.csv')


In [16]:
import re

df_results = []
# for action_string in list_action_strings:
for action_string in ["GPU.S3|buy-dismiss|DC3,1.0-96"]:
    action_comb = parse_action_string(action_string)
    solution_fleet = get_solution_from_action_comb(action_comb)
    solution_price_strategy = selling_prices.copy()
    solution_price_strategy.rename(columns={"selling_price": "price"}, inplace=True)
    solution_price_strategy['time_step'] = 1

    server_generation = action_comb[0]["server_generation"]
    latency_sensitivity = df_datacenters[df_datacenters['datacenter_id'] == action_comb[0]["datacenter_id"]]['latency_sensitivity'].iloc[0]

    server_capacity = df_servers[df_servers['server_generation'] == server_generation]['capacity'].iloc[0]
    average_U = action_comb[0]["average_U"]
    demand = pd.DataFrame({
        "time_step": list(range(1,168+1)),
        "server_generation": [server_generation]*168,
        "high": [0]*168,
        "medium": [0]*168,
        "low": [0]*168
    })
    demand[latency_sensitivity] = server_capacity * average_U
    score = evaluation_nofailure.get_evaluation(fleet=solution_fleet,
                                                pricing_strategy=solution_price_strategy,
                                                demand=demand,
                                            datacenters=datacenters,
                                            servers=servers,
                                            selling_prices=selling_prices,
                                            elasticity=elasticity,
                                            time_steps=168, 
                                            verbose=True,
                                            random_demand=False)
    real_score = evaluation_failure.get_evaluation(fleet=solution_fleet,
                                                pricing_strategy=solution_price_strategy,
                                                demand=demand,
                                            datacenters=datacenters,
                                            servers=servers,
                                            selling_prices=selling_prices,
                                            elasticity=elasticity,
                                            time_steps=168, 
                                            verbose=False,
                                            random_demand=False)
    print(action_string, score, real_score)
    df_results.append([action_string, score, real_score])
    
df_results = pd.DataFrame(df_results, columns=['action_string', 'score', 'real_score'])
df_results

latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 1, 'O': np.float64(-143921.25), 'P': np.float64(-143921.25)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 2, 'O': np.float64(-127650.0), 'P': np.float64(16271.25)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 3, 'O': np.float64(-111222.58), 'P': np.float64(16427.42)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 4, 'O': np.float64(-94662.58), 'P': np.float64(16560.0)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 5, 'O': np.float64(-77987.54), 'P': np.float64(16675.04)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 6, 'O': np.float64(-61211.45), 'P': np.float64(16776.09)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 7, 'O

,action_string,score,real_score
0,"GPU.S3|buy-dismiss|DC3,1.0-96",1.384690e+06,1.125490e+06


# buy-dismiss (using formula)

In [17]:
from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function, get_known, get_evaluation
import pandas as pd
import numpy as np

demand, datacenters, servers, selling_prices, elasticity = load_problem_data()
df_servers = pd.read_csv('./data/servers.csv')
df_datacenters = pd.read_csv('./data/datacenters.csv')
df_selling_price = pd.read_csv('./data/selling_prices.csv')


In [18]:
list_action_strings = []
for server_generation in df_servers['server_generation'].values:
    for datacenter_id in df_datacenters['datacenter_id'].values:
        for average_U in np.linspace(0.0, 1.0, 21)[1:]:
            life_expectancy = df_servers[df_servers['server_generation'] == server_generation]['life_expectancy'].iloc[0]
            for dismiss_age in range(life_expectancy, 0, -12):
                action_string = f"{server_generation}|buy-dismiss|{datacenter_id},{np.round(average_U,2)}-{dismiss_age}"
                list_action_strings.append(action_string)
len(list_action_strings)

4480

In [19]:
df_results = []
# for action_string in ["CPU.S1|buy-dismiss|DC1,0.6-24"]:
for action_string in list_action_strings:
    action_comb = parse_action_string(action_string)

    server_generation = action_comb[0]["server_generation"]
    datacenter_id = action_comb[0]["datacenter_id"]

    df_cur_server = df_servers[df_servers['server_generation'] == server_generation]
    df_cur_datacenter = df_datacenters[df_datacenters['datacenter_id'] == datacenter_id]

    cost_of_energy = df_cur_datacenter['cost_of_energy'].iloc[0]
    latency_sensitivity = df_cur_datacenter['latency_sensitivity'].iloc[0]

    selling_price = df_selling_price[np.logical_and(df_selling_price['server_generation'] == server_generation, 
                                                    df_selling_price['latency_sensitivity'] == latency_sensitivity)]['selling_price'].iloc[0]


    life_expectancy = df_cur_server['life_expectancy'].iloc[0]
    purchase_price = df_cur_server['purchase_price'].iloc[0]
    energy_consumption_per_day = df_cur_server['energy_consumption'].iloc[0]
    energy_cost_per_day = energy_consumption_per_day * cost_of_energy
    average_mantainance_cost_per_day = df_cur_server['average_maintenance_fee'].iloc[0]
    server_capacity = df_cur_server['capacity'].iloc[0]
    dismiss_age = action_comb[1]["dismiss_age"]
    failure_rate = 0.07260491698699582
    # failure_rate = 0.0
    average_U = action_comb[0]["average_U"]
    
    list_objective_per_day = []
    for i in range(1, dismiss_age+1):
        Z_fail = ((1-failure_rate) * server_capacity).astype(int)
        demand_met = np.minimum(Z_fail, average_U * server_capacity)
        U = demand_met / Z_fail
        L = i / life_expectancy
        R = demand_met * selling_price
        C = average_mantainance_cost_per_day * (1 + 1.5 * i / life_expectancy * np.log2(1.5 * i / life_expectancy)) + energy_cost_per_day
        if i == 1:
            C += purchase_price
        P = R - C
        list_objective_per_day.append([U, L, P])


    list_objective_per_day = np.asarray(list_objective_per_day)
    objective_per_day = list_objective_per_day[:,2]
    # objective_per_day = list_objective_per_day[:,0] * list_objective_per_day[:,1] * list_objective_per_day[:,2]
    total_objective = np.sum(objective_per_day)
    print(action_string, total_objective)
    df_results.append([action_string, total_objective])

CPU.S1|buy-dismiss|DC1,0.05-96 -46670.35643891591
CPU.S1|buy-dismiss|DC1,0.05-84 -39926.47822850248
CPU.S1|buy-dismiss|DC1,0.05-72 -34375.66425803408
CPU.S1|buy-dismiss|DC1,0.05-60 -29874.159171540658
CPU.S1|buy-dismiss|DC1,0.05-48 -26251.86531851012
CPU.S1|buy-dismiss|DC1,0.05-36 -23300.369534365585
CPU.S1|buy-dismiss|DC1,0.05-24 -20750.167311892517
CPU.S1|buy-dismiss|DC1,0.05-12 -18217.564610448822
CPU.S1|buy-dismiss|DC1,0.1-96 -43790.35643891591
CPU.S1|buy-dismiss|DC1,0.1-84 -37406.47822850248
CPU.S1|buy-dismiss|DC1,0.1-72 -32215.664258034078
CPU.S1|buy-dismiss|DC1,0.1-60 -28074.159171540658
CPU.S1|buy-dismiss|DC1,0.1-48 -24811.86531851013
CPU.S1|buy-dismiss|DC1,0.1-36 -22220.369534365585
CPU.S1|buy-dismiss|DC1,0.1-24 -20030.167311892517
CPU.S1|buy-dismiss|DC1,0.1-12 -17857.564610448822
CPU.S1|buy-dismiss|DC1,0.15-96 -40910.35643891591
CPU.S1|buy-dismiss|DC1,0.15-84 -34886.47822850248
CPU.S1|buy-dismiss|DC1,0.15-72 -30055.664258034078
CPU.S1|buy-dismiss|DC1,0.15-60 -26274.1591715406

In [20]:
df_results = pd.DataFrame(df_results, columns=['action_string', 'score'])
df_results['server_generation'] = df_results['action_string'].apply(lambda x: x.split("|")[0])
df_results = pd.DataFrame.join(df_results.set_index('server_generation'),
                  df_servers[['server_generation', 'slots_size']].set_index('server_generation'), 
                  on='server_generation', how='left').reset_index()
df_results['score_per_slot'] = df_results['score'] / df_results['slots_size']
df_results = df_results.sort_values(by=['score_per_slot', 'action_string'], ascending=False).reset_index(drop=True)
df_results.to_csv("resources/df_single_server_results_buydismiss_P.csv", index=False)
df_results

,server_generation,action_string,score,slots_size,score_per_slot
0,GPU.S3,"GPU.S3|buy-dismiss|DC3,1.0-96",1.125490e+06,4,281372.449799
1,GPU.S3,"GPU.S3|buy-dismiss|DC3,0.95-96",1.125490e+06,4,281372.449799
2,GPU.S3,"GPU.S3|buy-dismiss|DC3,0.9-96",1.125490e+06,4,281372.449799
3,GPU.S3,"GPU.S3|buy-dismiss|DC4,1.0-96",1.085170e+06,4,271292.449799
4,GPU.S3,"GPU.S3|buy-dismiss|DC4,0.95-96",1.085170e+06,4,271292.449799
...,...,...,...,...,...
4475,GPU.S2,"GPU.S2|buy-dismiss|DC4,0.05-96",-5.126764e+05,4,-128169.106426
4476,GPU.S3,"GPU.S3|buy-dismiss|DC4,0.1-96",-5.218702e+05,4,-130467.550201
4477,GPU.S3,"GPU.S3|buy-dismiss|DC4,0.05-84",-5.375715e+05,4,-134392.875819
4478,GPU.S3,"GPU.S3|buy-dismiss|DC3,0.05-96",-5.852302e+05,4,-146307.550201


# buy-move-dismiss

### simulation

In [26]:
import re

df_results = []
for action_string in ["GPU.S3|buy-move-dismiss|DC3,1.0-DC2,1.0,86-96"]:
    action_comb = parse_action_string(action_string)
    solution_fleet = get_solution_from_action_comb(action_comb)
    solution_price_strategy = selling_prices.copy()
    solution_price_strategy.rename(columns={"selling_price": "price"}, inplace=True)
    solution_price_strategy['time_step'] = 1
    
    server_generation = action_comb[0]["server_generation"]
    latency_sensitivity_1 = df_datacenters[df_datacenters['datacenter_id'] == action_comb[0]["datacenter_id"]]['latency_sensitivity'].iloc[0]
    latency_sensitivity_2 = df_datacenters[df_datacenters['datacenter_id'] == action_comb[1]["datacenter_id"]]['latency_sensitivity'].iloc[0]

    server_capacity = df_servers[df_servers['server_generation'] == server_generation]['capacity'].iloc[0]
    average_U = action_comb[0]["average_U"]
    demand = pd.DataFrame({
        "time_step": list(range(1,168+1)),
        "server_generation": [server_generation]*168,
        "high": [0]*168,
        "medium": [0]*168,
        "low": [0]*168
    })
    demand[latency_sensitivity_1] = server_capacity * average_U
    demand[latency_sensitivity_2] = server_capacity * average_U

    score = evaluation_nofailure.get_evaluation(fleet=solution_fleet,
                                                pricing_strategy=solution_price_strategy,
                                                demand=demand,
                                            datacenters=datacenters,
                                            servers=servers,
                                            selling_prices=selling_prices,
                                            elasticity=elasticity,
                                            time_steps=168, 
                                            verbose=True,
                                            random_demand=False)
    real_score = evaluation_failure.get_evaluation(fleet=solution_fleet,
                                                pricing_strategy=solution_price_strategy,
                                                demand=demand,
                                            datacenters=datacenters,
                                            servers=servers,
                                            selling_prices=selling_prices,
                                            elasticity=elasticity,
                                            time_steps=168, 
                                            verbose=False,
                                            random_demand=False)

    print(action_string, score, real_score)
    df_results.append([action_string, score, real_score])
df_results = pd.DataFrame(df_results, columns=['action_string', 'score', 'real_score'])
df_results

latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 1, 'O': np.float64(-143921.25), 'P': np.float64(-143921.25)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 2, 'O': np.float64(-127650.0), 'P': np.float64(16271.25)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 3, 'O': np.float64(-111222.58), 'P': np.float64(16427.42)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 4, 'O': np.float64(-94662.58), 'P': np.float64(16560.0)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 5, 'O': np.float64(-77987.54), 'P': np.float64(16675.04)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 6, 'O': np.float64(-61211.45), 'P': np.float64(16776.09)}
latency_sensitivity  high
server_generation        
GPU.S3                  8
{'time-step': 7, 'O

,action_string,score,real_score
0,"GPU.S3|buy-move-dismiss|DC3,1.0-DC2,1.0,86-96",1.376290e+06,1.119590e+06


### formula

In [27]:
from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function, get_known, get_evaluation
import pandas as pd
import numpy as np

demand, datacenters, servers, selling_prices, elasticity = load_problem_data()

df_servers = pd.read_csv('./data/servers.csv')
df_datacenters = pd.read_csv('./data/datacenters.csv')
df_selling_price = pd.read_csv('./data/selling_prices.csv')


In [28]:
df_servers_dict = df_servers.set_index('server_generation').to_dict('index')
df_datacenters_dict = df_datacenters.set_index('datacenter_id').to_dict('index')
df_selling_price_dict = df_selling_price.set_index(['server_generation', 'latency_sensitivity']).to_dict('index')

In [29]:
list_action_strings = []
for server_generation in df_servers['server_generation'].values:
    life_expectancy = df_servers_dict[server_generation]['life_expectancy']

    for datacenter_id_1 in df_datacenters['datacenter_id'].values:
        for datacenter_id_2 in df_datacenters['datacenter_id'].values:
            if datacenter_id_1 == datacenter_id_2:
                continue
            for average_U_1 in np.linspace(0.0, 1.0, 21)[1:]:
                for average_U_2 in np.linspace(0.0, 1.0, 21)[1:]:

                    for dismiss_age in range(life_expectancy, 0, -12):
                        for move_age in range(life_expectancy, 0, -12):
                            if move_age >= dismiss_age:
                                continue
                            action_string = f"{server_generation}|buy-move-dismiss|{datacenter_id_1},{np.round(average_U_1,2)}-{datacenter_id_2},{np.round(average_U_2,2)},{move_age}-{dismiss_age}"
                            list_action_strings.append(action_string)
len(list_action_strings)

940800

In [33]:
import concurrent.futures
from tqdm import tqdm


df_results = []
# for action_string in ['GPU.S3|buy-move-dismiss|DC3,1.0-DC2,1.0,86-96']:
for action_string in tqdm(list_action_strings):
    action_comb = parse_action_string(action_string)

    server_generation = action_comb[0]["server_generation"]
    datacenter_id_1 = action_comb[0]["datacenter_id"]
    datacenter_id_2 = action_comb[1]["datacenter_id"]

    average_U_1 = action_comb[0]["average_U"]
    average_U_2 = action_comb[1]["average_U"]

    df_cur_server = df_servers_dict[server_generation]
    df_cur_datacenter_1 = df_datacenters_dict[datacenter_id_1]
    df_cur_datacenter_2 = df_datacenters_dict[datacenter_id_2]

    cost_of_energy_1 = df_cur_datacenter_1['cost_of_energy']
    latency_sensitivity_1 = df_cur_datacenter_1['latency_sensitivity']

    cost_of_energy_2 = df_cur_datacenter_2['cost_of_energy']
    latency_sensitivity_2 = df_cur_datacenter_2['latency_sensitivity']


    selling_price_1 = df_selling_price_dict[(server_generation,latency_sensitivity_1)]['selling_price']
    selling_price_2 = df_selling_price_dict[(server_generation,latency_sensitivity_2)]['selling_price']

    energy_consumption_per_day = df_cur_server['energy_consumption']
    energy_cost_per_day_1 = energy_consumption_per_day * cost_of_energy_1
    energy_cost_per_day_2 = energy_consumption_per_day * cost_of_energy_2


    life_expectancy = df_cur_server['life_expectancy']
    purchase_price = df_cur_server['purchase_price']
    average_mantainance_cost_per_day = df_cur_server['average_maintenance_fee']
    server_capacity = df_cur_server['capacity']
    cost_of_moving = df_cur_server['cost_of_moving']

    move_age = action_comb[1]["move_age"]
    dismiss_age = action_comb[2]["dismiss_age"]
    # failure_rate = 0.0
    failure_rate = 0.07260491698699582
    
    list_objective_per_day = []
    for i in range(1, dismiss_age+1):
        selling_price = selling_price_1 if i < move_age + 1 else selling_price_2
        energy_cost_per_day = energy_cost_per_day_1 if i < move_age + 1 else energy_cost_per_day_2
        average_U = average_U_1 if i < move_age + 1 else average_U_2
        Z_fail = int((1-failure_rate) * server_capacity)
        demand_met = np.minimum(Z_fail, average_U * server_capacity)

        U = demand_met / Z_fail
        L = i / life_expectancy
        R = demand_met * selling_price
        C = average_mantainance_cost_per_day * (1 + 1.5 * i / life_expectancy * np.log2(1.5 * i / life_expectancy)) + energy_cost_per_day
        if i == 1:
            C += purchase_price
        if i == move_age + 1:
            C += cost_of_moving
        P = R - C
        # print(i, U, np.round(L,2), P)
        list_objective_per_day.append([U, L, P])


    list_objective_per_day = np.asarray(list_objective_per_day)
    objective_per_day = list_objective_per_day[:,2]
    total_objective = np.sum(objective_per_day)

    df_results.append([action_string, total_objective])
df_results = pd.DataFrame(df_results, columns=['action_string', 'score'])
df_results

100%|██████████| 940800/940800 [03:19<00:00, 4708.49it/s]


,action_string,score
0,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,84-96",-47970.356439
1,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,72-96",-48270.356439
2,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,60-96",-48570.356439
3,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,48-96",-48870.356439
4,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,36-96",-49170.356439
...,...,...
940795,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,24-48",520680.884788
940796,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,12-48",525720.884788
940797,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,24-36",349222.159146
940798,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,12-36",354262.159146


In [34]:
df_results = pd.DataFrame(df_results, columns=['action_string', 'score'])
df_results['server_generation'] = df_results['action_string'].apply(lambda x: x.split("|")[0])
df_results = pd.DataFrame.join(df_results.set_index('server_generation'),
                  df_servers[['server_generation', 'slots_size']].set_index('server_generation'), 
                  on='server_generation', how='left').reset_index()
df_results['score_per_slot'] = df_results['score'] / df_results['slots_size']
df_results = df_results.sort_values(by=['score_per_slot', 'action_string'], ascending=False).reset_index(drop=True)
df_results.to_csv("resources/df_single_server_results_buymovedismiss_P.csv", index=False)
df_results

,server_generation,action_string,score,slots_size,score_per_slot
0,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,12-96",1.119450e+06,4,279862.449799
1,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,0.95,12-96",1.119450e+06,4,279862.449799
2,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,0.9,12-96",1.119450e+06,4,279862.449799
3,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,0.95-DC3,1.0,12-96",1.119450e+06,4,279862.449799
4,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,0.95-DC3,0.95,12-96",1.119450e+06,4,279862.449799
...,...,...,...,...,...
940795,GPU.S3,"GPU.S3|buy-move-dismiss|DC3,0.05-DC4,0.05,36-96",-6.114302e+05,4,-152857.550201
940796,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,0.05-DC3,0.05,72-96",-6.164702e+05,4,-154117.550201
940797,GPU.S3,"GPU.S3|buy-move-dismiss|DC3,0.05-DC4,0.05,24-96",-6.164702e+05,4,-154117.550201
940798,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,0.05-DC3,0.05,84-96",-6.215102e+05,4,-155377.550201
